In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1. Загружаем мультиязычную модель
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. Определяем тексты для сравнения (можно на разных языках)
text1 = "Функция, которая позволяет очищать духовой шкаф без каких-либо действий со стороны пользователя. Закройте дверцу духовки и установите программу пиролитической очистки. Духовой шкаф заблокируется и начнет набирать температуру 480-500°С. При такой температуре весь жир, который отложился на стенках духового шкафа, сгорает и превращается в пепел. Программа длится около 3 часов. Как только температура в рабочей камере опустится ниже 180°С, дверца разблокируется. Вам остается только смахнуть салфеткой остатки пепла. Результат - абсолютно чистая эмаль без лишний усилий."
text2 = "Духовой шкаф Graude BMP 60.4 S оснащен пиролитической очисткой, при которой прибор заблокируется и начнет набирать температуру 480-500 °С. Весь жир, который отложился на стенках духового шкафа, сгорает и превращается в пепел. Вам остается только смахнуть салфеткой остатки пепла. Результат – абсолютно чистая эмаль без лишних усилий. Автоматические программы приготовления Pro Cook помогут с рецептами. Управление осуществляется с помощью TFT-дисплея высокого разрешения диагональю 7”. Противень AirFry гарантирует наибольший доступ циркулируемого горячего воздуха к еде для создания хрустящей корочки. Блюда, приготовленные в AirFry гораздо менее вредные, чем приготовленные в классической фритюрнице."

# 3. Преобразуем тексты в векторы (эмбеддинги)
embedding1 = model.encode(text1)
embedding2 = model.encode(text2)

# 4. Вычисляем косинусное сходство
similarity = util.cos_sim(embedding1, embedding2)

print(f"Степень сходства: {similarity.item():.4f}")


Степень сходства: 0.7895


In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1. Загружаем мультиязычную модель
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

path_text = "Размеченный текст.txt"
path_desc = "Описание на сайте.txt"

with open(path_text, "r", encoding="utf-8") as f:
    for line in f:
        print(line)


In [12]:
from sentence_transformers import SentenceTransformer, util

def find_unmatched_lines(file1_path: str, file2_path: str, threshold: float = 0.6):
    """
    Читает непустые строки из двух файлов и ищет для каждой строки из первого файла
    строку из второго со степенью сходства > threshold.
    Возвращает список строк из первого файла, для которых аналога не нашлось.
    """

    # Загружаем модель (однократно)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    # Читаем непустые строки
    def read_nonempty(path):
        with open(path, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]

    lines1 = read_nonempty(file1_path)
    lines2 = read_nonempty(file2_path)

    if not lines1:
        return []
    if not lines2:
        return lines1[:]  # если второй файл пуст — все строки unmatched

    # Эмбеддинги
    emb1 = model.encode(lines1, convert_to_tensor=True)
    emb2 = model.encode(lines2, convert_to_tensor=True)

    # Матрица косинусных сходств: shape [len(lines1), len(lines2)]
    sim_matrix = util.cos_sim(emb1, emb2)

    unmatched = []
    for i, line in enumerate(lines1):
        # Находим максимум по строке
        max_sim = float(sim_matrix[i].max())

        if max_sim <= threshold:
            unmatched.append(line)

    return unmatched

path_text = "Размеченный текст.txt"
path_desc = "Описание на сайте.txt"
unmatched = find_unmatched_lines(path_text, path_desc, threshold=0.4)
for line in unmatched:
    print(line)

Конвекция
Защита от детей
Телескопические направляющие
Полезный объем: 73 л
Ш х В х Г= 594 х 594 х 498 мм
Оснащение
Коллекция PREMIUM
Серия Premium – это минималистичный дизайн, отсутствие излишних декоративных элементов и лаконичный блок управления.
Стиль hi-tech в этой коллекции выражается не только во внешнем виде, но и в инновационном оснащении.


In [ ]:
from sentence_transformers import SentenceTransformer, util

def find_unmatched_with_text_search(file1_path: str, file2_path: str, threshold: float = 0.6):
    """
    Логика:
    1) Строки из первого файла без точки сначала ищутся во втором файле через полнотекстовый поиск.
    2) Если не найдены — идут в косинусное сравнение.
    3) Строки с точками сразу идут в косинусное сравнение.
    4) Возвращаются строки, для которых аналогов не найдено.
    """

    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    # Чтение непустых строк
    def read_nonempty(path):
        with open(path, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]

    lines1 = read_nonempty(file1_path)
    lines2 = read_nonempty(file2_path)

    if not lines1:
        return []
    if not lines2:
        return lines1[:]  # если второй файл пуст — все unmatched

    # Эмбеддинги второго файла (один раз)
    emb2 = model.encode(lines2, convert_to_tensor=True)

    unmatched = []

    for line in lines1:
        # ---- 1. ПОЛНОТЕКСТОВЫЙ ПОИСК ДЛЯ СТРОК БЕЗ ТОЧЕК ----
        if '.' not in line:
            found = False
            for l2 in lines2:
                # Вариант A: подстрока (строгий и быстрый)
                if line.lower() in l2.lower():
                    found = True
                    break

                # Вариант B (если нужно строгое совпадение)
                # if line.lower() == l2.lower():
                #    found = True
                #    break

            if found:
                continue  # аналог найден полнотекстово — сразу к следующей строке

        # ---- 2. КОСИНУСНОЕ СРАВНЕНИЕ ----
        emb1 = model.encode(line, convert_to_tensor=True)
        sims = util.cos_sim(emb1, emb2)  # 1 × len(lines2)
        max_sim = float(sims.max())

        if max_sim <= threshold:
            unmatched.append(line)

    return unmatched
